In [1]:
import sys
from pathlib import Path
parent_dir = Path.cwd().parent
sys.path.append(str(parent_dir))
sys.path.append('./../../RoboPoint/')

from robopoint_utils import load_model, get_coordinates, generate_prompt, do_inference_with_logits, calculate_probs_per_coordinate
from utils import extract_all, plot_scatter, calculate_euclidian_distance, calculate_normalized_euclidian_distance, visualize_points_on_image, close_all_images, cluster_data, calculate_spread, show_cluster, calculate_roc_curve, plot_roc_curve, find_best_threshold
from PIL import Image

2025-02-27 12:43:56.289933: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-27 12:43:56.328685: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-27 12:43:56.328715: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-27 12:43:56.328736: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-27 12:43:56.336684: I tensorflow/core/platform/cpu_feature_g

In [2]:
model_name = "wentao-yuan/robopoint-v1-vicuna-v1.5-13b"
model, processor, tokenizer = load_model(model_name)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Some weights of the model checkpoint at wentao-yuan/robopoint-v1-vicuna-v1.5-13b were not used when initializing LlavaLlamaForCausalLM: ['model.vision_tower.vision_tower.vision_model.embeddings.class_embedding', 'model.vision_tower.vision_tower.vision_model.embeddings.patch_embedding.weight', 'model.vision_tower.vision_tower.vision_model.embeddings.position_embedding.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.layer_norm1.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.layer_norm1.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.layer_norm2.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.layer_norm2.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.mlp.fc1.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.mlp.fc1.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.mlp.fc2.bias', 'model.vision_tower.vision_tower.vision_model.encoder

In [3]:
def generate_vqa_prompt(prompt):
    text = '<image>' + '\n' + prompt
    conv = conv_templates["llava_v1"].copy()
    conv.append_message(conv.roles[0], text)
    conv.append_message(conv.roles[1], None)
    return conv.get_prompt()

In [12]:
import numpy as np

def run_vqa_experiment(prompts, temperatures, runs_per_instance, epsilon, min_samples, ignoreCoordinateDecimalPlaces, experiment_path = "../data/first_experiments"):
    tasks, images, ground_truths, object_list = extract_all(experiment_path)
    for i in range(len(tasks)):
        print(f"Task: {i + 1}")
        print(tasks[i])
    
        image_path = images[i]
        task = tasks[i]
        objects = object_list[i]
        groundtruths = ground_truths[i]

        with Image.open(image_path) as image:
            width, height = image.size
            for object, groundtruth in zip(objects, groundtruths):

                prompt_sample = "Your task is to locate the following object in the picture: " + object + ". Decide between the following coordinates: [(0.8, 0.2), (0.6, 0.4), (0.5, 0.5)] " + "Your answer should be one coordinate, i.e. (x1, y1). The coordinates should are between 0 and 1, indicating the normalized pixel locations of the points in the image."
                prompt = generate_prompt(object, "Your task is to locate the following object in the picture: ", "Decide between the following coordinates: [(0.8, 0.2), (0.6, 0.4), (0.5, 0.5)]. Your answer should be one coordinate, i.e. (x1, y1). The coordinates should are between 0 and 1, indicating the normalized pixel locations of the points in the image.")
                for temperature in temperatures:
                
                    for j in range(runs_per_instance):
                        generated_text, outputs = do_inference_with_logits(image, prompt, model, processor, tokenizer, temperature)
                        print(f"output with temp {temperature}: {generated_text}")

In [13]:
PROMPTS = [
    "Locate several points within the space of the ",
    "Locate several points within the area of the ",
    "Identify multiple points within the space of the ",
    "Identify several points within the area of the ",
    "Identify several spots within the space of the "
]

TEMPERATURES = [
    0.6
]


run_vqa_experiment(
    prompts = PROMPTS,
    temperatures = TEMPERATURES,
    runs_per_instance = 1,
    epsilon = 20,
    min_samples = 3,
    ignoreCoordinateDecimalPlaces = True,
    experiment_path = "../data/base_experiments_fast"
)

Task: 1
Place the pot to the right of the blue fork.
A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions. USER: <image>
Your task is to locate the following object in the picture: pot. Decide between the following coordinates: [(0.8, 0.2), (0.6, 0.4), (0.5, 0.5)]. Your answer should be one coordinate, i.e. (x1, y1). The coordinates should are between 0 and 1, indicating the normalized pixel locations of the points in the image. ASSISTANT:
output with temp 0.6: [(0.41, 0.350), (0.46, 0.298), (0.40, 0.308), (0.38, 0.377), (0.48, 0.388), (0.48, 0.340), (0.44, 0.431), (0.45, 0.256), (0.49, 0.273), (0.52, 0.310), (0.42, 0.231), (0.53, 0.362), (0.38, 0.273), (0.43, 0.390), (0.37, 0.335), (0.51, 0.417)]
A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions. USER: <image>
Your task is to loc

KeyboardInterrupt: 